# 1. Import Libraries

In [ ]:
# Basic Imports
import os
import sys
import io

import numpy as np
import pandas as pd
import datetime

import requests
import zipfile
import shutil

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

path_prefix = "../"

import xml.etree.ElementTree as ET

sys.path.insert(0, os.path.abspath(path_prefix))

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# Special Imports
import cv2

import tensorflow as tf

import keras

import keras.backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.engine.topology import Layer
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Activation, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# Project Imports
from config import path_config

np.random.seed(100)

# 2. Define required Classes and Methods

## 2.1. Local Response Normalization

In [ ]:
class LocalResponseNormalization(Layer):

    def __init__(self, n = 5, alpha = 0.0005, beta = 0.75, k = 2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x, mask=None):

        _, r, c, f = self.shape
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1, 1), padding="same", pool_mode="avg")

        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)

        denom = K.pow(self.k + averaged, self.beta)
        return x / denom

    def get_output_shape_for(self, input_shape):
        return input_shape

## 2.2. Wrapper class for selecting Model Type

In [ ]:
class WrapperModel:
    """
    Wrapper class for Model Selection
    """

    def __init__(self, height, width, channels):
        """

        :param height:
        :param width:
        :param channels:
        """

        self.height = height
        self.width = width
        self.channels = channels

    def get_model(self):
        """

        :return:
        """
        model = None

        if path_config.MODEL_NAME == 'MAlexNet':
            model = MAlexNetModel(self.height, self.width, self.channels)
        elif path_config.MODEL_NAME == 'AlexNet':
            model = AlexNetModel(self.height, self.width, self.channels)
        elif path_config.MODEL_NAME == 'InceptionV4':
            model = InceptionV4(self.height, self.width, self.channels)

        return model

## 2.3. M-AlexNet Model

In [ ]:
class MAlexNetModel:
    """
    mAlexNet model in Keras
    """

    def __init__(self, height, width, channels):
        """

        :param height:
        :param width:
        :param channels:
        """

        self.height = height
        self.width = width
        self.channels = channels

    def build_model(self):
        """

        :return:
        """

        model = Sequential()
        model.add(
            Conv2D(16, (11, 11), padding='same', strides=(4, 4), input_shape=(self.height, self.width, self.channels),
                   name='mAlex_conv1'))
        model.add(Activation('relu', name='mAlex_relu1'))
        model.add(LocalResponseNormalization(name='mAlex_norm1'))
        model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='mAlex_pool1'))

        model.add(Conv2D(20, (5, 5), padding='same', strides=(1, 1), name='mAlex_conv2'))
        model.add(Activation('relu', name='mAlex_relu2'))
        model.add(LocalResponseNormalization(name='mAlex_norm2'))
        model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='mAlex_pool2'))

        model.add(Conv2D(30, (3, 3), padding='same', strides=(1, 1), name='mAlex_conv3'))
        model.add(Activation('relu', name='mAlex_relu3'))
        model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='mAlex_pool3'))

        model.add(Flatten(name='mAlex_flatten'))

        model.add(Dense(48, activation='relu', name='mAlex_fc4'))
        model.add(Dense(1, activation='sigmoid', name='mAlex_fc5'))

        return model

## 2.4. AlexNet Model

In [ ]:
class AlexNetModel:
    """
    AlexNet model in Keras
    """

    def __init__(self, height, width, channels):
        """

        :param height:
        :param width:
        :param channels:
        """
        self.height = height
        self.width = width
        self.channels = channels

    def build_model(self):
        """

        :return:
        """

        model = Sequential()
        model.add(
            Conv2D(96, (11, 11), padding='same', strides=(4, 4), input_shape=(self.height, self.width, self.channels),
                   name='Alex_conv1'))
        model.add(Activation('relu', name='Alex_relu1'))
        model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='Alex_pool1'))
        model.add(LocalResponseNormalization(name='Alex_norm1'))

        model.add(Conv2D(256, (5, 5), padding='same', strides=(1, 1), name='Alex_conv2'))
        model.add(Activation('relu', name='Alex_relu2'))
        model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='Alex_pool2'))
        model.add(LocalResponseNormalization(name='Alex_norm2'))

        model.add(Conv2D(384, (3, 3), padding='same', strides=(1, 1), name='Alex_conv3'))
        model.add(Activation('relu', name='Alex_relu3'))

        model.add(Conv2D(384, (3, 3), padding='same', strides=(1, 1), name='Alex_conv4'))
        model.add(Activation('relu', name='Alex_relu4'))

        model.add(Conv2D(256, (3, 3), padding='same', strides=(1, 1), name='Alex_conv5'))
        model.add(Activation('relu', name='Alex_relu5'))
        model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='Alex_pool5'))
        model.add(Dropout(rate=0.5, name='Alex_dropout5'))

        model.add(Flatten(name='Alex_flatten'))

        model.add(Dense(units=4096, activation='relu', name='Alex_fc6'))
        model.add(Dropout(rate=0.5, name='Alex_dropout6'))

        model.add(Dense(units=4096, activation='relu', name='Alex_fc7'))

        model.add(Dense(1, activation='sigmoid', name='mAlex_fc8'))

        return model

## 2.5. InceptionV4 Model

In [ ]:
class InceptionV4:
    """
        Inception v4 model in Keras
        """

    def __init__(self, height, width, channels):
        """

        :param height:
        :param width:
        :param channels:
        """

        self.height = height
        self.width = width
        self.channels = channels

    def build_model(self):
        """

        :return:
        """

        model = Sequential()

        return model

# 3. Train Model

In [ ]:
def model_train_generator(train_type, path_prefix):
    start_time = datetime.datetime.now()

    model_obj = None
    
    # Import model graph
    if train_type == 'rgb':
        model_obj = WrapperModel(path_config.IMG_HEIGHT, path_config.IMG_WIDTH, path_config.N_CHANNELS)
    elif train_type == 'grayscale':
        model_obj = WrapperModel(path_config.IMG_HEIGHT, path_config.IMG_WIDTH, 1)
    
    # Create instance of DL Model
    model = model_obj.get_model()
    
    # Build the DL Model
    model = model.build_model()
    
    # Pring DL Model Summary
    print(model.summary())
    
    # Create instance of Image Data Generator
    train_datagen = ImageDataGenerator(
        horizontal_flip=True,
        rescale=1. / 255,
        validation_split=0.20
    )

    train_generator = train_datagen.flow_from_directory(
        path_prefix + path_config.PATH_DATA_PROCESSED + path_config.PATH_DATA_COMBINED,
        subset='training',
        target_size=(path_config.IMG_HEIGHT, path_config.IMG_WIDTH),
        color_mode='rgb',
        batch_size=path_config.BATCH_SIZE_TRAIN,
        class_mode='binary',
        classes=['empty', 'occupied'],
        # save_to_dir = '../../Datasets/Augmented/training/',
        # save_format='jpeg',
        seed=100)

    validation_generator = train_datagen.flow_from_directory(
        path_prefix + path_config.PATH_DATA_PROCESSED + path_config.PATH_DATA_COMBINED,
        subset='validation',
        target_size=(path_config.IMG_HEIGHT, path_config.IMG_WIDTH),
        color_mode='rgb',
        batch_size=path_config.BATCH_SIZE_VALIDATION,
        class_mode='binary',
        classes=['empty', 'occupied'],
        # save_to_dir='../../Datasets/Augmented/validation/',
        # save_format='jpeg',
        seed=100)

    # Optimizer
    model_sgd = SGD(lr=path_config.LEARNING_RATE, 
                    decay=path_config.WEIGHT_DECAY_RATE, 
                    momentum=path_config.MOMENTUM,
                    nesterov=path_config.FLAG_NESTEROV)

    # Callbacks
    model_tb = TensorBoard(
        log_dir=path_prefix + path_config.PATH_LOGS_TENSORBOARD + datetime.datetime.now().strftime(
            '%Y-%m-%d_%H-%M-%S') + '_epoch_' + str(
            path_config.EPOCHS) + '_batch_' + str(path_config.BATCH_SIZE_TRAIN),
        histogram_freq=0, write_graph=True, write_images=True)
    
    model_checkpoint = ModelCheckpoint(
        filepath=path_prefix + path_config.PATH_WEIGHTS + path_config.WEIGHTS_NAME + '-checkpoint_' + start_time.strftime(
            '%Y-%m-%d_%H-%M') + '_' + '{epoch:02d}-{val_loss:.2f}.h5',
        verbose=1)
    
    print('Model training begins...')
    
    # Training model
    model.compile(loss='binary_crossentropy', optimizer=model_sgd, metrics=['accuracy'])

    model.fit_generator(
        train_generator,
        steps_per_epoch=path_config.STEPS_TRAIN,
        epochs=path_config.EPOCHS,
        validation_data=validation_generator,
        validation_steps=path_config.STEPS_VALIDATION,
        callbacks=[model_checkpoint, model_tb])

    print('Model training completed.')
    
    model_weight_filename = path_prefix + path_config.PATH_MODEL + path_config.MODEL_NAME + '_' + train_type + '_generator_' + start_time.strftime(
        '%Y-%m-%d_%H-%M') + '.h5'

    print('Saving trained model weights...')
    model.save_weights(model_weight_filename)
    
    end_time = datetime.datetime.now()
    print('Total Time taken to train the model: ' + str((end_time - start_time).total_seconds() / 60.0))

    return model_weight_filename

# 4. Test Model

In [ ]:
def show_comparison(origin_image, transformed_image):
    plt.figure(figsize=(15,12))
    plt.subplot(121), plt.imshow(origin_image, cmap='gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(transformed_image, cmap='gray')
    plt.title('Transformed Image'), plt.xticks([]), plt.yticks([])
    plt.show()

class ModelPredict:
    def __init__(self):
        """

        """

        model_obj = WrapperModel(path_config.IMG_HEIGHT, path_config.IMG_WIDTH, path_config.N_CHANNELS)
        model = model_obj.get_model()
        model = model.build_model()

        gray_model_obj = WrapperModel(path_config.IMG_HEIGHT, path_config.IMG_WIDTH, 1)
        gray_model = gray_model_obj.get_model()
        gray_model = gray_model.build_model()

        self.model = model
        self.gray_model = gray_model

    def model_predict_rgb(self, model_weight_filepath):
        """

        :param model_weight_filepath:
        :return:
        """
        model = self.model
        model.load_weights(model_weight_filepath)
        sample_image = path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000001.jpg'
        im = cv2.imread(sample_image)

        x_shift = 130
        y_shift = 300
        x_range = [100, 300, 550, 750]
        y_range = [100, 600]

        im = cv2.resize(im, (1000, 1000))
        im = np.array(im)

        images = []
        for x in x_range:
            for y in y_range:
                im_ = im[x + 2:x + x_shift - 2, y + 2:y + y_shift - 2]
                im_ = cv2.resize(im_, (path_config.IMG_WIDTH, path_config.IMG_HEIGHT))
                im_ = np.array(im_)
                # im_ = im_.transpose(1, 0, 2)
                images.append(im_)

        images = np.array(images)

        predictions = model.predict(images)
        predictions = np.hstack(predictions > path_config.ACCURACY_THRESHOLD).astype(int)

        i = 0
        im_ = np.copy(im)
        for x in x_range:
            for y in y_range:
                im_ = cv2.rectangle(im_, (y, x + 2), (y + y_shift, x + x_shift - 2),
                                    (255 * int(predictions[i]), 255 * int(1 - predictions[i]), 0), 2)
                i += 1

        show_comparison(im, im_)

    def crop_and_rotate(self, img, cnt, rect, box):
        """

        :param img:
        :param cnt:
        :param rect:
        :param box:
        :return:
        """
        W = rect[1][0]
        H = rect[1][1]

        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)

        angle = rect[2]

        if W >= H:
            angle = angle + 90
            croppedW = H
            croppedH = W
        else:
            croppedW = W
            croppedH = H

        # Center of rectangle in source image
        center = ((x1 + x2) / 2, (y1 + y2) / 2)

        '''
        # Size of the upright rectangle bounding the rotated rectangle
        size = (x2 - x1, y2 - y1)
        M = cv2.getRotationMatrix2D((size[0] / 2, size[1] / 2), angle, 1.0)
    
        # Cropped upright rectangle
        cropped = cv2.getRectSubPix(img, size, center)
        cropped = cv2.warpAffine(cropped, M, size, borderMode = cv2.BORDER_CONSTANT)
    
        # Final cropped & rotated rectangle
        croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW), int(croppedH)), (size[0] / 2, size[1] / 2))
        '''

        cnt = np.array([cnt[0], cnt[1], cnt[3], cnt[2]], np.float32)
        window = np.array([[0, 0], [croppedW, 0], [0, croppedH], [croppedW, croppedH]], np.float32)
        M = cv2.getPerspectiveTransform(cnt, window)
        cropped = cv2.warpPerspective(img, M, (int(croppedW), int(croppedH)))

        return cropped

    def model_predict_rgb_xml(self, model_weight_filepath, sample_image_path, sample_image_xml_path, thickness=2,
                              grayscale_flag=False):
        """

        :param model_weight_filepath:
        :param sample_image_path:
        :param sample_image_xml_path:
        :param thickness:
        :param grayscale_flag:
        :return:
        """
        if grayscale_flag:
            model = self.gray_model
        else:
            model = self.model
        model.load_weights(model_weight_filepath)

        pklot_pupcr_xml_tree = ET.parse(sample_image_xml_path)
        pklot_pupcr_xml_root = pklot_pupcr_xml_tree.getroot()

        im = cv2.imread(sample_image_path)
        im = np.array(im)
        image_name = (sample_image_path.split('.')[-2]).split('/')[-1]

        contour_display_image = im.copy()

        for space in pklot_pupcr_xml_root:
            for value in space:
                if value.tag == 'contour':
                    cnt = []
                    for contour in value:
                        cnt.append([[int(contour.attrib['x']), int(contour.attrib['y'])]])

                    cnt = np.array(cnt)
                    rect = cv2.minAreaRect(cnt)
                    # print(rect)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    # print(box)
                    im_ = self.crop_and_rotate(im, cnt, rect, box)
                    im_ = cv2.resize(im_, (path_config.IMG_WIDTH, path_config.IMG_HEIGHT))
                    im_ = np.array(im_)
                    cv2.imwrite(path_prefix + path_config.PATH_DATA_TRANSFORMED + image_name + str(space.attrib.get('id')) + '.jpg',
                                im_)

                    im_ = im_ / 255.0
                    # show_comparison(im, im_)

                    if grayscale_flag:
                        im_ = get_grayscale(im_)
                        im_ = apply_gaussian_mask(im_)
                        im_ = get_fft(im_)
                        im_ = cv2.normalize(im_, None, 0, 1, cv2.NORM_MINMAX)
                        im_ = np.expand_dims(im_, axis=2)

                    im_ = np.expand_dims(im_, axis=0)
                    prediction = model.predict(im_)
                    # print('ORIGINAL: ', prediction)

                    if prediction >= path_config.ACCURACY_THRESHOLD:
                        prediction = 1
                    else:
                        prediction = 0

                    # print('MODIFIED: ', prediction)
                    # print('\n')

                    cv2.drawContours(contour_display_image, [cnt], 0,
                                     (0, 255 * int(1 - prediction), 255 * int(prediction)),
                                     thickness)

        cv2.imwrite(path_prefix + path_config.PATH_DATA_VALIDATION_RESULTS + image_name + '_res' + '.jpg',
                    contour_display_image)

        plt.imshow(contour_display_image, cmap='gray')
        plt.xticks([])
        plt.yticks([])
        plt.show()

    def model_predict_ensemble(self, model_weight_filepath_rgb, model_weight_filepath_grayscale, sample_image_path,
                               sample_image_xml_path, thickness=2):
        """

        :param model_weight_filepath_rgb:
        :param model_weight_filepath_grayscale:
        :param sample_image_path:
        :param sample_image_xml_path:
        :param thickness:
        :return:
        """
        model1 = self.model
        model2 = self.gray_model

        model1.load_weights(model_weight_filepath_rgb)
        model2.load_weights(model_weight_filepath_grayscale)

        pklot_pupcr_xml_tree = ET.parse(sample_image_xml_path)
        pklot_pupcr_xml_root = pklot_pupcr_xml_tree.getroot()

        im = cv2.imread(sample_image_path)
        im = np.array(im)

        contour_display_image = im.copy()

        for space in pklot_pupcr_xml_root:
            for value in space:
                if value.tag == 'contour':
                    cnt = []
                    for contour in value:
                        cnt.append([[int(contour.attrib['x']), int(contour.attrib['y'])]])

                    cnt = np.array(cnt)
                    rect = cv2.minAreaRect(cnt)
                    # print(rect)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    # print(box)
                    im_ = self.crop_and_rotate(im, cnt, rect, box)
                    im_ = cv2.resize(im_, (path_config.IMG_WIDTH, path_config.IMG_HEIGHT))
                    im_ = np.array(im_)

                    image_rgb = np.expand_dims(im_, axis=0)
                    prediction_rgb = model1.predict(image_rgb)

                    im_ = get_grayscale(im_)
                    im_ = apply_gaussian_mask(im_)
                    im_ = get_fft(im_)
                    im_ = cv2.normalize(im_, None, 0, 1, cv2.NORM_MINMAX)
                    im_ = np.expand_dims(im_, axis=2)
                    image_grayscale = np.expand_dims(im_, axis=0)
                    prediction_grayscale = model2.predict(image_grayscale)

                    prediction = max([prediction_rgb, prediction_grayscale])

                    if prediction >= path_config.ACCURACY_THRESHOLD:
                        prediction = 1
                    else:
                        prediction = 0

                    cv2.drawContours(contour_display_image, [box], 0,
                                     (255 * int(prediction), 255 * int(1 - prediction), 0),
                                     thickness)

        plt.imshow(contour_display_image, cmap='gray')
        plt.xticks([])
        plt.yticks([])
        plt.show()

# 5. Check Tensorboard logs

#### http://localhost:6006

# 6. Execute/Start Model Training

In [ ]:
grayscale_flag = False

if grayscale_flag:
    model_type = 'grayscale'
else:
    model_type = 'rgb'

In [ ]:
model_weight_filename = path_prefix + path_config.PATH_MODEL + path_config.MODEL_NAME + '_rgb_generator_224_224.h5'

In [ ]:
if not os.path.exists(model_weight_filename):
    model_weight_filename = model_train_generator(model_type, path_prefix)
    pass

# 7. Execute/Start Model Prediction

In [ ]:
model_predictor = ModelPredict()

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000002.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000002.xml', 2,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000003.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000003.xml', 2,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000004.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000004.xml', 2,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000005.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000005.xml', 2,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000006.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000006.xml', 2,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000007.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000007.xml', 7,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000008.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000008.xml', 6,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000009.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000009.xml', 6,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000010.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000010.xml', 6,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000011.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000011.xml', 2,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000012.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000012.xml', 6,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000013.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000013.xml', 6,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000014.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000014.xml', 6,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000015.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000015.xml', 6,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000017.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000017.xml', 6,
                                      grayscale_flag)

In [ ]:
plt.figure(figsize=(20, 12))

model_predictor.model_predict_rgb_xml(model_weight_filename,
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000019.jpg',
                                      path_prefix + path_config.PATH_DATA_VALIDATION + 'validation_0000019.xml', 6,
                                      grayscale_flag)